# 1. 环境配置

## 1.1 python 环境准备

In [24]:
! pip install gradio==6.1.0 openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 beautifulsoup4==4.14.3 langchain_chroma==1.1.0 arxiv==2.3.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/90/7f/340847023184305a6378d75ec71e1dd38a942dfe71b7c29314b8fbe26948/arxiv-2.3.1-py3-none-any.whl (11 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/4e/eb/c96d64137e29ae17d83ad2552470bafe3a7a915e85434d9942077d7fd011/feedparser-6.0.12-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl

   -------------------------- ------------- 2/3 [arxiv]
   ---------------------------------------- 3/3 [arxiv]



## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 LangSmith 环境配置
我们需要先前往 LangSmith 的官网并进行注册登录。

登录后我们就进入了下面这个初始界面，此时我们需要找到左下角的 Setting ，然后在里面先获取新建一个 API Key。

创建完成后，我们就可以将其配置到环境变量中。除了 API_Key 以外，通常 LangSmith 的项目还需要设置是否跟踪、上传地址以及项目名称信息（这个需要自定义设置）。

In [3]:
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "your langsmith api key"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "ai-studio-tracing"

# 2. LangSmith Tracing

## 2.1 wrap_openai() 

### 2.1.1 单次调用

假如我们的目的只是单独跟踪应用中使用的大模型的调用情况的话，可以使用 LangSmith 提供 wrap_openai() 来自动包装 OpenAI SDK，让所有 API 调用自动记录。

目前绝大部分的模型都支持 OpenAI 格式进行调用，包括我们前面讲到的通义千问以及 AI Studio 中的所有模型：

我们只需要导入 openai 库后创建客户端 OpenAI()，并传入 api_key 以及 base_url 两个变量即可创建客户端等待后续的调用：

In [4]:
from openai import OpenAI
import os

raw_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://aistudio.baidu.com/llm/lmapi/v3"
)

为了能够顺利的上传到 LangSmith 上，我们需要使用 LangSmith 里的 wrappers 来进行一层包裹，从而能够让这个模型调用的信息能够实时同步到 LangSmith 中：

In [5]:
from langsmith.wrappers import wrap_openai
traced_client = wrap_openai(raw_client)

在包裹完后，我们就可以使用包裹后的客户端进行调用了并打印结果了：

In [6]:
response = traced_client.chat.completions.create(
    model="ernie-3.5-8k",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "用一句话介绍一下你自己。"}
    ]
)

print(response.choices[0].message.content)

我是能陪你畅聊、答疑、学习，还能帮你处理各种事务的智能助手。


### 2.1.2 添加元数据或标签

假如我们希望往里再额外添加一些元数据或标签的话，可以在 wrap_openai 中添加内容：

In [7]:
from langsmith.wrappers import wrap_openai
traced_client = wrap_openai(raw_client, tracing_extra={"metadata": {"my-key": "my-value"}, "tags": ["a-tag"]})

response = traced_client.chat.completions.create(
    model="ernie-3.5-8k",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "用一句话介绍一下你自己。"}
    ]
)

print(response.choices[0].message.content)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


我是一个能帮你答疑解惑、陪你谈天说地，还能提供各种实用建议的智能助手。


又或者是在 chat_completion 中添加相关内容：

In [8]:
from langsmith.wrappers import wrap_openai
traced_client = wrap_openai(raw_client)

response = traced_client.chat.completions.create(
 model="ernie-3.5-8k",
 messages=[
 {"role": "system", "content": "You are a helpful assistant."},
 {"role": "user", "content": "用一句话介绍一下你自己。"}], 
 langsmith_extra={ "tags": ["my-other-tag"], "metadata": {"my-other-key": "my-value"}, },
)

print(response.choices[0].message.content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


我是能陪你畅快聊天、答疑解惑、提供各类实用信息的智能助手。


### 2.1.3 多次调用并对比
在 LangSmith 里除了监控单次的调用以外，其还有一个很重要的能力就是对比多次调用并找到最优解。对于 AI Studio 里调用的模型而言，本质上就是修改调用时候的 model 参数（具体的模型信息可到 AI Studio-帮助文档 中查询）。

比如这里修改为 DeepSeek-Chat 这个模型（输入参数应该是 deepseek-v3） ：

In [9]:
from langsmith.wrappers import wrap_openai
traced_client = wrap_openai(raw_client)

response = traced_client.chat.completions.create(
    model="deepseek-v3",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "用一句话介绍一下你自己。"}])

print(response.choices[0].message.content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


我是一款由深度求索公司打造的AI助手，致力于用简洁、准确的语言为你提供各类问题的解答和实用帮助。


## 2.2 @traceable

除了通过 Wrap_openai 监控大模型调用外，我们还可以将大模型调用转换成函数并使用LangSmith 中的 @traceable 装饰器实现动态的跟踪（也要配置 LangSmith 环境）：

In [10]:
from openai import OpenAI
from langsmith import traceable 

@traceable()
def baidu(query):
    # 创建 OpenAI Client
    raw_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),
        base_url="https://aistudio.baidu.com/llm/lmapi/v3")
    # 调用大模型
    response = raw_client.chat.completions.create(
        model="ernie-3.5-8k",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}])
    return response.choices[0].message.content

假如此时对该函数进行调用并获取回复：

In [11]:
print(baidu("用一句话介绍一下你自己。"))

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


我是能帮你答疑解惑、陪你畅快聊天、还能辅助你创作各类文本的智能小助手呀。


在这个装饰器里，我们可以添加几个常用的参数，比如说：
- name：设置到时候在 LangSmith 上显示的名称
- metadata：添加部分元数据，比如直接在 @traceable 里添加参数 metadata={"vectordb": "sklearn"}，或者可以在调用函数时添加 baidu(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
- run_type：运行时展示的类型，这个具体影响在 LangSmith 上显示的信息（没有设置 run_type 时默认使用 Chain 类型）

比如对于我们模型调用这个函数，我们就可以为其添加上这两个参数：

In [12]:
@traceable(name="baidu_ernie", run_type="llm")
def baidu(query):
    # 创建 OpenAI Client
    raw_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),
        base_url="https://aistudio.baidu.com/llm/lmapi/v3")
    # 调用大模型
    response = raw_client.chat.completions.create(
        model="ernie-3.5-8k",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}])
    return response.choices[0].message.content

假如此时对该函数再次进行调用：

In [13]:
print(baidu("用一句话介绍一下你自己。"))

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


我是一个能陪你畅快聊天、为你答疑解惑、助力创作各类文本的智能小助手。


除了大模型调用可以跟踪以外，其实所有我们想要跟踪的函数都可以进行跟踪。比如说一段简单的打招呼函数和乘法函数：

In [14]:
@traceable(name="multiply_numbers")
def multiply(a, b):
    return a * b

@traceable(name="greet_user")
def greet(name):
    return f"你好，{name}！欢迎使用 LangSmith 追踪功能。"

那假如我们希望将其组合起来获取输出结果的话，我们也可以设计一个组合的函数，并对其进行追踪：

In [15]:
@traceable(name="workflow_example")
def workflow(name):
    # 追踪普通函数
    msg = greet(name)
    result = multiply(len(name), 3)

    # 追踪 LLM 调用
    messages=f"{msg} 请评价一下数字 {result}。"
    llm_answer = baidu(messages)

    return {
        "greet": msg,
        "multiply_result": result,
        "llm_answer": llm_answer
    }

假如此时对该函数再次进行调用：

In [16]:
print(workflow("李老师"))

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


{'greet': '你好，李老师！欢迎使用 LangSmith 追踪功能。', 'multiply_result': 9, 'llm_answer': '李老师，这数字9啊，它就像个挺特别的存在呢。从形状上看，它弯弯绕绕的，仿佛在诉说着一种独特的故事。在数学里，它可是个不小的数字，比8大，又比10小，处于一个中间偏上的位置。而且啊，9在很多文化里都有着特别的寓意，比如在中国文化里，9常常和长久、尊贵联系在一起，象征着一种持久和崇高的品质。所以呀，这数字9，不仅有着独特的外观，还蕴含着丰富的文化内涵呢！'}


## 2.3 自动跟踪

对于非 LangChain 原生的代码而言，我们需要使用添加 @traceable 来实现 LangSmith 的跟踪。但是对于 LangChain 的原生代码而言，其实我们只需要配置好 LangSmith 的环境变量即可自动实现跟踪。

### 2.3.1 LCEL 链

比如下面这段使用 LCEL 进行链式调用的代码，我们可以先定义一个大模型：

In [17]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

然后定义一个提示词模版：

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
解析以下产品评价中的信息，按格式要求输出：
{format_instructions}
评价：{input}""")

在然后定义一个格式化输出的 parser：

In [19]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class ProductInfo(BaseModel):
 product: str = Field(description="产品名称")
 price: float = Field(description="产品价格")
 rating: float = Field(description="评分（1-5的数字）")
parser = JsonOutputParser(pydantic_object=ProductInfo)
format_instructions = parser.get_format_instructions()

最后用 LCEL 的链将其串起来并进行调用：

In [20]:
chain = prompt | llm | parser
response = chain.invoke({"input": "这款无线耳机很好用，价格399元，我给4.5星好评",
  "format_instructions": format_instructions})
print("结构化结果：", response)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


结构化结果： {'product': '无线耳机', 'price': 399, 'rating': 4.5}


### 2.3.2 RAG 系统

对于 RAG 和 Agent 的系统也是一样的，只要我们在之前的代码上面加上 LangSmith 的环境配置，即可详细的查看到内部的情况。

比如对于 RAG 系统而言，需要先创建数据库：

In [21]:
# RAG_db_create.py
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://zh.d2l.ai/chapter_introduction/index.html")
docs = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1500,
  chunk_overlap = 150)
splits = text_splitter.split_documents(docs)

from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os
embeddings = DashScopeEmbeddings(
  dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
  model="text-embedding-v1")
vectordb = Chroma.from_documents(documents=splits,
  embedding=embeddings,
  persist_directory='./chroma')

USER_AGENT environment variable not set, consider setting it to identify your requests.
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


然后再对数据库进行调用：

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

embeddings = DashScopeEmbeddings(
  dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
  model="text-embedding-v1")
vectordb = Chroma(persist_directory="./chroma", embedding_function=embeddings)
template = """请使用以下上下文信息回答最后的问题。如果您不知道答案，就直接说您不知道，不要试图编造答案。回答最多使用三句话。请尽可能简洁地回答。最后一定要说“谢谢提问！”。
上下文：{context}
问题：{question}
有帮助的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
 openai_api_key=os.environ.get("OPENAI_API_KEY"),
 base_url="https://aistudio.baidu.com/llm/lmapi/v3")
retriever = vectordb.as_retriever(search_type="mmr", 
 search_kwargs={"k": 1, "fetch_k": 10, "lambda_mult": 0.25})
def format_docs(docs):
 return "\n\n".join(doc.page_content for doc in docs)
qa_chain = (
 {"context": retriever | format_docs, "question": RunnablePassthrough()}
 | QA_CHAIN_PROMPT
 | llm
 | StrOutputParser())
print(qa_chain.invoke("日常生活里，哪里用到了机器学习呢？"))

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


日常生活中，语音识别（如唤醒词检测、语音转文字）、地图路线推荐等用到了机器学习。这些应用通过分析数据或交互经验，自动提升性能。谢谢提问！


### 2.3.5 Agent 系统

又比如对于 Agent 系统而言，我们可以先创建好智能体再调用（此时我们可以在调用时传入 thread_id）：

In [25]:
from langchain_community.chat_models import ChatTongyi
import os
llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")

from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

from langchain.agents import create_agent
agent = create_agent(model=llm, 
                     tools=tools, 
                     system_prompt="You are a helpful assistant", 
                     checkpointer=memory)

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


论文编号 1605.08386 的信息如下：

- **发布日期**：2016-05-26
- **标题**：Heat-bath random walks with Markov bases
- **作者**：Caprice Stanley, Tobias Windisch
- **摘要**：研究了由有限允许移动集生成的格点图，这些图的边可以是任意长度。我们证明了在固定整数矩阵的纤维上，这些图的直径可以从上面由一个常数来限制。然后我们研究了这些图上的热浴随机游走的混合行为。我们还给出了移动集的显式条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度下是一个扩展器。


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## 2.4 Thread 线程

除了在模型调用时候添加 thread_id 外，对于添加 @traceable 装饰器实现模型跟踪的方式也可以添加 thread_id ，这个就是在调用函数的时候添加上一些元数据即可，比如：

In [ ]:
print(workflow("李老师",langsmith_extra={"metadata": {"thread_id": "user_1"}}))

{'greet': '你好，李老师！欢迎使用 LangSmith 追踪功能。', 'multiply_result': 9, 'llm_answer': '数字9呀，它可是个很有意思的数字呢！在数学里，9是最大的个位数，还是3的倍数，有很多特别的性质。在中国文化里，9还常常和长久、圆满联系在一起，象征着好运和幸福。你觉得9怎么样呢？是不是也觉得它很特别呀？'}


我们可以点击右上角的 Thread 按钮或在 Tracing 界面的上方就可以看到同一 thread_id 的对话信息。